In [2]:
import os
import numpy as np
import pandas as pd
import math
import warnings
import time as time

In [3]:
points = pd.read_csv(os.path.join("data","points.csv"), sep=",")

In [4]:
#Clase default de funciones para el algoritmo
class EmptyFunction:
    def fun(x):      return 0
    def grad(x):     return 0
    def prox(x,tau): return x
    
#Clase para la función W(x,y) de Weber denotada por f
class f:
    #Constructor
    def __init__(self,imagen= 0, proximal = 0):
        self.imagen = imagen
        self.proximal = proximal

    #Evaluación de la función
    def fun(self ,x): 
        self.imagen = 0
        for i in range(0,len(points)):
            x_i = np.array([points["x"][i], points["y"][i]])
            self.imagen += points["peso"][i]*np.linalg.norm(x-x_i)**2
        return self.imagen
    
    #Operador proximal para la función de Weber
    def prox(self, x, tau):
        self.proximal = 0
        denominador = 2 * points["peso"].sum() - (1/tau)
        suma_x = 0
        suma_y = 0
        for i in range(0,len(points)):
            suma_x+= 2 * points["peso"][i] * points["x"][i]
            suma_y+= 2 * points["peso"][i] * points["y"][i]
        numerador_x = suma_x - (1/tau)*x[0]
        numerador_y = suma_y - (1/tau)*x[1]
        self.proximal = np.array([numerador_x / denominador, numerador_y / denominador] )
        return self.proximal

#Clase para la función g de distancia de un punto a la recta asociada a la ruta 68   
class g:
    
    #Constructor
    def __init__(self,imagen= 0, gradiente = 0):
        self.imagen = imagen
        self.gradiente = gradiente
    
    #Evaluación de la función
    def fun(self, x):
        self.imagen = 0
        numerador=128 * x[0] + 553 * x[1] + 155777
        denominador=np.sqrt(128**2 + 553**2)
        if numerador >0 :
            self.imagen = numerador / denominador
            return self.imagen
        if numerador < 0: 
            self.imagen = (-numerador / denominador)
            return self.imagen
        
    #Gradiente de la función 
    def grad(self, x):
        self.gradiente = 0
        numerador=128 * x[0] + 553 * x[1] + 155777
        denominador= np.sqrt(128**2 + 553**2)
        gradient = np.array([0.0,0.0])
        if numerador > 0 :
            gradient[0] = 128 / denominador
            gradient[1] = 553 / denominador
        if numerador < 0: 
            gradient[0] = (-128) / denominador
            gradient[1] = (-553) / denominador
        self.gradiente = gradient
        return self.gradiente
        
        

def MGP(x_init, f=None, g=None, opt=None):
    # default inputs
    if f   is None: f = EmptyFunction()
    if g   is None: g = EmptyFunction()
    if opt is None: opt = {'tol': 1e-4, 'iter': 500}

    # Parametros algoritmicos
    tol      = opt['tol']  #Tolerancia
    max_iter = opt['iter'] #Nro maximo de iteraciones
    
    # paso (en este caso constante)
    tau = 1.0
    
    # inicialización 
    x = x_init

    print('Running MGP...');
    
    timing = np.zeros(max_iter) #array de tiempo por iteracion
    criter = np.zeros(max_iter) #Array de evaluación funcion objetivo por iteracion

    # Loop algoritmico
    for it in range(max_iter):
    
        t = time.time()
    
        # Actualizacion de x_k:= x en cada iteración 
        x_old = x;
        x = x - tau * ( g.grad(x))
        x = f.prox(x, tau);
    

        # Actualizacion de tiempo de ejecucion y de funcion objetivo
        timing[it] = time.time() - t
        criter[it] = f.fun(x) + g.fun(x)
           
        # Criterio de parada
        if np.linalg.norm(x - x_old) < tol * np.linalg.norm(x_old):
            break
    
        print( str(it) + ' de un maximo de ' + str(max_iter) + ' iteraciones' )

    criter = criter[0:it+1]
    timing = np.cumsum(timing[0:it+1])
    
    return x, it, timing, criter

f = f()
g = g()
MGP(np.array([0,0]), f,g)

Running MGP...
0 de un maximo de 500 iteraciones
1 de un maximo de 500 iteraciones
2 de un maximo de 500 iteraciones
3 de un maximo de 500 iteraciones


(array([385.87786274, 234.05272394]),
 4,
 array([0.00096607, 0.00096607, 0.00196314, 0.00296092, 0.00296092]),
 array([329792.17438296, 324460.27017925, 324448.81850181, 324448.70503658,
        324448.70898955]))